<a href="https://colab.research.google.com/github/shiyuhu1933/EC-523-final-project/blob/main/resnet50_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import torch
import os
import h5py
import numpy as np
import torch.nn as nn

drive.mount('/content/gdrive')

DATASET_GOOGLE_DRIVE_PATH = '/content/gdrive/MyDrive/Deep_Learning /data_hdf5_100_10'


def get_file_name(files_list):
  with open(files_list) as f:
    return [line.rstrip()[:] for line in f]

class FaceMaskData(torch.utils.data.Dataset):

  def __init__(self, mode, config, transform=None):

    self.data_dir = DATASET_GOOGLE_DRIVE_PATH
    self.transform = transform

    if mode == 'train':
      self.files = get_file_name(os.path.join(self.data_dir, 'train_files.txt'))
    else:
      self.files = get_file_name(os.path.join(self.data_dir, 'test_files.txt'))

    image = []
    label = []

    for dataset in self.files:
      path = os.path.join(self.data_dir, dataset)
      self.file = h5py.File(path, 'r')
      self.total_num_imgs, self.H, self.W, self.C = self.file['image'].shape
      image.append(self.file['image'][:])
      label.append(self.file['labels'][:])

    self.image= np.vstack(image)
    self.label = np.vstack(label)

    self.num_images = len(self.image) 
    self.num_classes = len(np.unique(self.label))

  def __getitem__(self, index):
    """Return one image and its corresponding attribute label."""
    image = self.image[index]
    label = self.label[index]
    if self.transform:
        image = self.transform(image)
    return image, torch.FloatTensor(label)

  def __len__(self):
    return self.num_images
      
  def get_num_class(self):
    return self.num_classes

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
def get_configuration(repeat_num, batch_size):
    config = {}

    config['device'] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    config['batch_size'] = batch_size
    config['num_workers'] = 1
    config['repeat_num'] = repeat_num
    config['lr'] = 0.0001

    return config


def image_to_rgb(images):
  imgs = []
  for i in range(len(images)):
    image_r = torch.unsqueeze(images[i,:,:,0], 0)
    image_g = torch.unsqueeze(images[i,:,:,1], 0)
    image_b = torch.unsqueeze(images[i,:,:,2], 0)
    image_split_rgb = torch.cat((image_r, image_g, image_b), 0)
    image_split_rgb = torch.unsqueeze(image_split_rgb, 0)
    imgs.append(image_split_rgb)
  return torch.cat(imgs)


def print_network(model, name):
  """Print out the network information."""
  num_params = 0
  print("\n")
  print("model name", name)
  print(model)
  num_params = sum([par.numel() for par in model.parameters()])
  print("The number of parameters: {}".format(num_params))

def save_model(model, step):
  save_model_path = "/content/gdrive/MyDrive/Deep Learning /models"
  model_path = os.path.join(save_model_path, 'model-{}.ckpt'.format(step))
  torch.save(model, model_path)
  print('Saved model checkpoints into {}...'.format(save_model_path))

def load_model(iters):
  path = "/content/gdrive/MyDrive/Deep Learning /models"
  model_path = os.path.join(path, 'model-{}.ckpt'.format(iters))
  model = torch.load(model_path)
  return model

In [3]:
import torch.nn as nn
import torchvision.models as models

config = get_configuration(repeat_num=10000, batch_size=512)
dataset = FaceMaskData('train', config)
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                              batch_size=config['batch_size'],
                              shuffle=True, 
                              num_workers=config['num_workers'])
num_classes = dataset.get_num_class()

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
   
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

'''Reference: https://pytorch.org/hub/pytorch_vision_resnet/'''

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=num_classes, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model


def resnet50(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)




In [4]:
model = resnet50().cuda()
print_network(model, 'iresnet50')



model name iresnet50
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_s

In [7]:
import torch
import torch.nn as nn
import math

class ArcFaceloss(nn.Module):

    def __init__(self, s=45.0, m=0.1, weight = None):
        super(ArcFaceloss, self).__init__()
      
        self.weight = weight
        self.s = s
        self.cosm = math.cos(m)
        self.sinm = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
    
    def forward(self, input, labels):
        
        cost = F.normalize(input)
        sint = torch.sqrt(1.0 - torch.square(cost))
        cosmt = self.s * (self.cosm * cost - self.sinm * sint)
        k = torch.where(cost > self.th, cosmt, self.s * (cost - self.mm))
        
        label = torch.zeros_like(cost)
        label.scatter_(1,labels.view(-1,1).long(),1)
        output = (1 - label) * self.s * cost + label * k
        
        cross_entropy = nn.CrossEntropyLoss()
        output = cross_entropy(output, labels)
        # print(output)

        return output 

In [8]:
import time
from torch import nn, optim
import torchvision.models as models
import torch.nn.functional as F
import datetime

model = resnet50().cuda()

criterion = ArcFaceloss().cuda()

optimizer = optim.SGD(model.parameters(), lr=0.05, momentum =0.9, weight_decay=5e-4)

Epochs = 30

loss = []

print('Start training ...')
start_time = time.time()
for epoch in range(1, Epochs+1):
  model.train()

  loss_acc = 0 
  for i, data in enumerate(data_loader, 0):
    images, labels = data
    images, labels = images.float().cuda(), labels.cuda()
    images = image_to_rgb(images)
    labels = labels.type(torch.LongTensor).cuda()
    labels = labels.squeeze(-1)
    # print(images.size())
    # print(labels.size())
    logits = model(images)
    # print(logits.size())
    loss = criterion(logits, labels)
    loss_acc += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  loss_acc /= Epochs
  et = time.time() - start_time
  et = str(datetime.timedelta(seconds=et))[:-7]
  log = "Elapsed [{}], Iteration [{}/{}]".format(et, epoch, Epochs)
  log += ', Loss = %.8f'%(loss_acc)
  print(log)

  # if epoch % 5 == 0:
  #   save_model(model, epoch)

Start training ...
Elapsed [0:01:05], Iteration [1/30], Loss = 31.31382627
Elapsed [0:02:11], Iteration [2/30], Loss = 30.99222415
Elapsed [0:03:17], Iteration [3/30], Loss = 30.81356258
Elapsed [0:04:22], Iteration [4/30], Loss = 30.63004990
Elapsed [0:05:28], Iteration [5/30], Loss = 30.35018520
Elapsed [0:06:34], Iteration [6/30], Loss = 30.01006432
Elapsed [0:07:39], Iteration [7/30], Loss = 29.59962896
Elapsed [0:08:45], Iteration [8/30], Loss = 29.19376933
Elapsed [0:09:51], Iteration [9/30], Loss = 28.82907769
Elapsed [0:10:56], Iteration [10/30], Loss = 28.16470420
Elapsed [0:12:02], Iteration [11/30], Loss = 26.96114823
Elapsed [0:13:08], Iteration [12/30], Loss = 25.34369539
Elapsed [0:14:13], Iteration [13/30], Loss = 22.98560268
Elapsed [0:15:19], Iteration [14/30], Loss = 20.00484832
Elapsed [0:16:25], Iteration [15/30], Loss = 16.38940900
Elapsed [0:17:30], Iteration [16/30], Loss = 13.08472748
Elapsed [0:18:36], Iteration [17/30], Loss = 9.61037385
Elapsed [0:19:42], Ite

In [9]:
dataset_test = FaceMaskData('test', config)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test,
                              batch_size=config['batch_size'],
                              shuffle=True, 
                              num_workers=config['num_workers'])
def test_on_resnet(model, test_loader):
    
    ## -- ! code required  
    num_correct = 0
    total = 0
    

    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data

            images, labels = images.float().cuda(), labels.cuda()
            images = image_to_rgb(images)
            labels = labels.type(torch.LongTensor).cuda()
            labels = labels.squeeze(-1)
        
            logits = model(images)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            num_correct += (predicted == labels).sum().item()
    acc = 100 * num_correct / total         
    return acc

acc = test_on_resnet(model, test_loader)
print('Accuracy of the network on the 10000 test images: %d %%' % (acc))

Accuracy of the network on the 10000 test images: 89 %
